In [13]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/workspace/RNN/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/workspace/RNN


In [14]:
!pip install docopt
! pip install sacrebleu

In [15]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import torch.nn as nn

class ModelEmbeddings(nn.Module):
    """
    Class that converts input words to their embeddings.
    """
    def __init__(self, embed_size, vocab):
        """
        Init the Embedding layers.

        @param embed_size (int): Embedding size (dimensionality)
        @param vocab (Vocab): Vocabulary object containing src and tgt languages
                              See vocab.py for documentation.
        """
        super(ModelEmbeddings, self).__init__()
        self.embed_size = embed_size

        # default values
        self.source = None
        self.target = None

        src_pad_token_idx = vocab.src['<pad>']
        tgt_pad_token_idx = vocab.tgt['<pad>']

        self.source = nn.Embedding(len(vocab.src), self.embed_size, padding_idx = src_pad_token_idx)
        self.target = nn.Embedding(len(vocab.tgt), self.embed_size, padding_idx = tgt_pad_token_idx)



In [16]:
import sys
from collections import namedtuple
from typing import Dict, List, Set, Tuple, Union

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
Hypothesis = namedtuple('Hypothesis', ['value', 'score'])
class NMT(nn.Module):
    """ Simple Neural Machine Translation Model:
        - RNN Encoder
        - Unidirection LSTM Decoder
        - Global Attention Model (Luong, et al. 2015)
    """
    def __init__(self, embed_size, hidden_size, vocab, dropout_rate=0.2, rnn_layer=nn.LSTM, num_layers=1, activation=torch.tanh,bidirectional=False):
        """ Init NMT Model.

        @param embed_size (int): Embedding size (dimensionality)
        @param hidden_size (int): Hidden Size, the size of hidden states (dimensionality)
        @param vocab (Vocab): Vocabulary object containing src and tgt languages
                              See vocab.py for documentation.
        @param dropout_rate (float): Dropout probability, for attention
        """
        super(NMT, self).__init__()
        self.model_embeddings = ModelEmbeddings(embed_size, vocab)
        self.hidden_size = hidden_size
        self.dropout_rate = dropout_rate
        self.vocab = vocab
        self.rnn_layer = rnn_layer
        self.num_layers = num_layers
        self.activation = activation

        # default values
        self.encoder = None
        self.decoder = None
        self.h_projection = None
        self.c_projection = None
        self.att_projection = None
        self.combined_output_projection = None
        self.target_vocab_projection = None
        self.dropout = None

        # model layers
        self.is_lstm = (rnn_layer == nn.LSTM)
        self.encoder = rnn_layer(input_size=embed_size, hidden_size=hidden_size, bidirectional=bidirectional, bias=True, num_layers=num_layers)
        self.decoder = nn.LSTMCell(input_size=embed_size+hidden_size, hidden_size=hidden_size, bias=True)
        self.h_projection = nn.Linear(hidden_size*2, hidden_size, bias=False)
        if self.is_lstm:
            self.c_projection = nn.Linear(hidden_size*2, hidden_size, bias=False)
        self.att_projection = nn.Linear(hidden_size*2, hidden_size, bias=False)
        self.combined_output_projection = nn.Linear(hidden_size*3, hidden_size, bias=False)
        self.target_vocab_projection = nn.Linear(hidden_size, len(vocab.tgt), bias=False)
        self.dropout = nn.Dropout(p=self.dropout_rate)


    def forward(self, source: List[List[str]], target: List[List[str]]) -> torch.Tensor:
        """ Take a mini-batch of source and target sentences, compute the log-likelihood of
        target sentences under the language models learned by the NMT system.

        @param source (List[List[str]]): list of source sentence tokens
        @param target (List[List[str]]): list of target sentence tokens, wrapped by `<s>` and `</s>`

        @returns scores (Tensor): a variable/tensor of shape (b, ) representing the
                                    log-likelihood of generating the gold-standard target sentence for
                                    each example in the input batch. Here b = batch size.
        """
        # Compute sentence lengths
        source_lengths = [len(s) for s in source]

        # Convert list of lists into tensors
        source_padded = self.vocab.src.to_input_tensor(source, device=self.device)   # Tensor: (src_len, b)
        target_padded = self.vocab.tgt.to_input_tensor(target, device=self.device)   # Tensor: (tgt_len, b)

        enc_hiddens, dec_init_state = self.encode(source_padded, source_lengths)
        enc_masks = self.generate_sent_masks(enc_hiddens, source_lengths)
        combined_outputs = self.decode(enc_hiddens, enc_masks, dec_init_state, target_padded)
        P = F.log_softmax(self.target_vocab_projection(combined_outputs), dim=-1)

        # Zero out, probabilities for which we have nothing in the target text
        target_masks = (target_padded != self.vocab.tgt['<pad>']).float()

        # Compute log probability of generating true target words
        target_gold_words_log_prob = torch.gather(P, index=target_padded[1:].unsqueeze(-1), dim=-1).squeeze(-1) * target_masks[1:]
        scores = target_gold_words_log_prob.sum(dim=0)
        return scores


    def encode(self, source_padded: torch.Tensor, source_lengths: List[int]) -> Tuple[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        """ Apply the encoder to source sentences to obtain encoder hidden states.
            Additionally, take the final states of the encoder and project them to obtain initial states for decoder.

        @param source_padded (Tensor): Tensor of padded source sentences with shape (src_len, b), where
                                        b = batch_size, src_len = maximum source sentence length. Note that
                                       these have already been sorted in order of longest to shortest sentence.
        @param source_lengths (List[int]): List of actual lengths for each of the source sentences in the batch
        @returns enc_hiddens (Tensor): Tensor of hidden units with shape (b, src_len, h*2), where
                                        b = batch size, src_len = maximum source sentence length, h = hidden size.
        @returns dec_init_state (tuple(Tensor, Tensor)): Tuple of tensors representing the decoder's initial
                                                hidden state and cell.
        """
        enc_hiddens, dec_init_state = None, None
        last_hidden, last_cell = None, None
        X = self.model_embeddings.source(source_padded) # (src_len, b, e)
        enc_hiddens, last_state = self.encoder(pack_padded_sequence(X, source_lengths))
        if self.is_lstm:
            last_hidden, last_cell = last_state
        else:
            last_hidden = last_state
        enc_hiddens, _ = pad_packed_sequence(enc_hiddens) # returns seq_unpacked, lens_unpacked
        enc_hiddens = enc_hiddens.permute(1, 0, 2)

        init_decoder_hidden = self.h_projection(torch.cat((last_hidden[-2], last_hidden[-1]), 1))

        if self.is_lstm:
            # (b, 2*h)
            init_decoder_cell = self.c_projection(torch.cat((last_cell[-2], last_cell[-1]), 1))
            dec_init_state = (init_decoder_hidden, init_decoder_cell)
        else:
            dec_init_state = (init_decoder_hidden, torch.zeros_like(init_decoder_hidden))

        print(enc_hiddens.shape)
        print("*"*40)
        return enc_hiddens, dec_init_state


    def decode(self, enc_hiddens: torch.Tensor, enc_masks: torch.Tensor,
                dec_init_state: Tuple[torch.Tensor, torch.Tensor], target_padded: torch.Tensor) -> torch.Tensor:
        """Compute combined output vectors for a batch.

        @param enc_hiddens (Tensor): Hidden states (b, src_len, h*2), where
                                     b = batch size, src_len = maximum source sentence length, h = hidden size.
        @param enc_masks (Tensor): Tensor of sentence masks (b, src_len), where
                                     b = batch size, src_len = maximum source sentence length.
        @param dec_init_state (tuple(Tensor, Tensor)): Initial state and cell for decoder
        @param target_padded (Tensor): Gold-standard padded target sentences (tgt_len, b), where
                                       tgt_len = maximum target sentence length, b = batch size.

        @returns combined_outputs (Tensor): combined output tensor  (tgt_len, b,  h), where
                                        tgt_len = maximum target sentence length, b = batch_size,  h = hidden size
        """
        # Chop of the <END> token for max length sentences.
        target_padded = target_padded[:-1]

        # Initialize the decoder state (hidden and cell)
        dec_state = dec_init_state

        # Initialize previous combined output vector o_{t-1} as zero
        batch_size = enc_hiddens.size(0)
        o_prev = torch.zeros(batch_size, self.hidden_size, device=self.device)

        # Initialize a list we will use to collect the combined output o_t on each step
        combined_outputs = []

        enc_hiddens_proj = self.att_projection(enc_hiddens) # (b, src_len, h)
        Y = self.model_embeddings.target(target_padded) # (tgt_len, b, e)
        for Y_t in torch.split(Y, 1, dim=0):
            Y_t = torch.squeeze(Y_t, 0) # (b, e)
            Ybar_t = torch.cat((Y_t, o_prev), 1)
            dec_state, o_t, e_t = self.step(Ybar_t, dec_state, enc_hiddens, enc_hiddens_proj, enc_masks)
            combined_outputs.append(o_t)
            o_prev = o_t
        combined_outputs = torch.stack(combined_outputs, dim=0) # (tgt_len, b, h)

        return combined_outputs


    def step(self, Ybar_t: torch.Tensor,
            dec_state: Tuple[torch.Tensor, torch.Tensor],
            enc_hiddens: torch.Tensor,
            enc_hiddens_proj: torch.Tensor,
            enc_masks: torch.Tensor) -> Tuple[Tuple, torch.Tensor, torch.Tensor]:
        """ Compute one forward step of the LSTM decoder, including the attention computation.

        @param Ybar_t (Tensor): Concatenated Tensor of [Y_t o_prev], with shape (b, e + h). The input for the decoder,
                                where b = batch size, e = embedding size, h = hidden size.
        @param dec_state (tuple(Tensor, Tensor)): Tuple of tensors both with shape (b, h), where b = batch size, h = hidden size.
                First tensor is decoder's prev hidden state, second tensor is decoder's prev cell.
        @param enc_hiddens (Tensor): Encoder hidden states Tensor, with shape (b, src_len, h * 2), where b = batch size,
                                    src_len = maximum source length, h = hidden size.
        @param enc_hiddens_proj (Tensor): Encoder hidden states Tensor, projected from (h * 2) to h. Tensor is with shape (b, src_len, h),
                                    where b = batch size, src_len = maximum source length, h = hidden size.
        @param enc_masks (Tensor): Tensor of sentence masks shape (b, src_len),
                                    where b = batch size, src_len is maximum source length.

        @returns dec_state (tuple (Tensor, Tensor)): Tuple of tensors both shape (b, h), where b = batch size, h = hidden size.
                First tensor is decoder's new hidden state, second tensor is decoder's new cell.
        @returns combined_output (Tensor): Combined output Tensor at timestep t, shape (b, h), where b = batch size, h = hidden size.
        @returns e_t (Tensor): Tensor of shape (b, src_len). It is attention scores distribution.
                                Note: You will not use this outside of this function.
                                      We are simply returning this value so that we can sanity check
                                      your implementation.
        """

        combined_output = None
        dec_state = self.decoder(Ybar_t, dec_state)
        dec_hidden, dec_cell = dec_state
        e_t = torch.squeeze(torch.bmm(enc_hiddens_proj, torch.unsqueeze(dec_hidden, dim=2)), dim=2)

        # Set e_t to -inf where enc_masks has 1 to ignore <pad> tokens
        if enc_masks is not None:
            e_t.data.masked_fill_(enc_masks.bool(), -float('inf'))

        alpha_t = F.softmax(e_t, dim=1) # (b, src_len)
        a_t = torch.squeeze(torch.bmm(torch.unsqueeze(alpha_t, 1), enc_hiddens), dim=1) # (b, 2h)
        U_t = torch.cat((dec_hidden, a_t), dim=1)
        V_t = self.combined_output_projection(U_t)
        O_t = self.dropout(self.activation(V_t))

        combined_output = O_t
        return dec_state, combined_output, e_t

    def generate_sent_masks(self, enc_hiddens: torch.Tensor, source_lengths: List[int]) -> torch.Tensor:
        """ Generate sentence masks for encoder hidden states.

        @param enc_hiddens (Tensor): encodings of shape (b, src_len, 2*h), where b = batch size,
                                     src_len = max source length, h = hidden size.
        @param source_lengths (List[int]): List of actual lengths for each of the sentences in the batch.

        @returns enc_masks (Tensor): Tensor of sentence masks of shape (b, src_len),
                                    where src_len = max source length, h = hidden size.
        """
        enc_masks = torch.zeros(enc_hiddens.size(0), enc_hiddens.size(1), dtype=torch.float)
        for e_id, src_len in enumerate(source_lengths):
            enc_masks[e_id, src_len:] = 1
        return enc_masks.to(self.device)


    def beam_search(self, src_sent: List[str], beam_size: int=5, max_decoding_time_step: int=70) -> List[Hypothesis]:
        """ Given a single source sentence, perform beam search, yielding translations in the target language.
        @param src_sent (List[str]): a single source sentence (words)
        @param beam_size (int): beam size
        @param max_decoding_time_step (int): maximum number of time steps to unroll the decoding RNN
        @returns hypotheses (List[Hypothesis]): a list of hypothesis, each hypothesis has two fields:
                value: List[str]: the decoded target sentence, represented as a list of words
                score: float: the log-likelihood of the target sentence
        """
        src_sents_var = self.vocab.src.to_input_tensor([src_sent], self.device)

        src_encodings, dec_init_vec = self.encode(src_sents_var, [len(src_sent)])
        src_encodings_att_linear = self.att_projection(src_encodings)

        h_tm1 = dec_init_vec
        att_tm1 = torch.zeros(1, self.hidden_size, device=self.device)

        eos_id = self.vocab.tgt['</s>']

        hypotheses = [['<s>']]
        hyp_scores = torch.zeros(len(hypotheses), dtype=torch.float, device=self.device)
        completed_hypotheses = []

        t = 0
        while len(completed_hypotheses) < beam_size and t < max_decoding_time_step:
            t += 1
            hyp_num = len(hypotheses)

            exp_src_encodings = src_encodings.expand(hyp_num,
                                                     src_encodings.size(1),
                                                     src_encodings.size(2))

            exp_src_encodings_att_linear = src_encodings_att_linear.expand(hyp_num,
                                                                           src_encodings_att_linear.size(1),
                                                                           src_encodings_att_linear.size(2))

            y_tm1 = torch.tensor([self.vocab.tgt[hyp[-1]] for hyp in hypotheses], dtype=torch.long, device=self.device)
            y_t_embed = self.model_embeddings.target(y_tm1)

            x = torch.cat([y_t_embed, att_tm1], dim=-1)

            (h_t, cell_t), att_t, _  = self.step(x, h_tm1,
                                                      exp_src_encodings, exp_src_encodings_att_linear, enc_masks=None)

            # log probabilities over target words
            log_p_t = F.log_softmax(self.target_vocab_projection(att_t), dim=-1)

            live_hyp_num = beam_size - len(completed_hypotheses)
            contiuating_hyp_scores = (hyp_scores.unsqueeze(1).expand_as(log_p_t) + log_p_t).view(-1)
            top_cand_hyp_scores, top_cand_hyp_pos = torch.topk(contiuating_hyp_scores, k=live_hyp_num)

            prev_hyp_ids = top_cand_hyp_pos // len(self.vocab.tgt)
            hyp_word_ids = top_cand_hyp_pos % len(self.vocab.tgt)

            new_hypotheses = []
            live_hyp_ids = []
            new_hyp_scores = []

            for prev_hyp_id, hyp_word_id, cand_new_hyp_score in zip(prev_hyp_ids, hyp_word_ids, top_cand_hyp_scores):
                prev_hyp_id = prev_hyp_id.item()
                hyp_word_id = hyp_word_id.item()
                cand_new_hyp_score = cand_new_hyp_score.item()

                hyp_word = self.vocab.tgt.id2word[hyp_word_id]
                new_hyp_sent = hypotheses[prev_hyp_id] + [hyp_word]
                if hyp_word == '</s>':
                    completed_hypotheses.append(Hypothesis(value=new_hyp_sent[1:-1],
                                                           score=cand_new_hyp_score))
                else:
                    new_hypotheses.append(new_hyp_sent)
                    live_hyp_ids.append(prev_hyp_id)
                    new_hyp_scores.append(cand_new_hyp_score)

            if len(completed_hypotheses) == beam_size:
                break

            live_hyp_ids = torch.tensor(live_hyp_ids, dtype=torch.long, device=self.device)
            h_tm1 = (h_t[live_hyp_ids], cell_t[live_hyp_ids])
            att_tm1 = att_t[live_hyp_ids]

            hypotheses = new_hypotheses
            hyp_scores = torch.tensor(new_hyp_scores, dtype=torch.float, device=self.device)

        if len(completed_hypotheses) == 0:
            completed_hypotheses.append(Hypothesis(value=hypotheses[0][1:],
                                                   score=hyp_scores[0].item()))

        completed_hypotheses.sort(key=lambda hyp: hyp.score, reverse=True)

        return completed_hypotheses
    @property
    def device(self) -> torch.device:
        """ Determine which device to place the Tensors upon, CPU or GPU.
        """
        return self.model_embeddings.source.weight.device

    @staticmethod
    def load(model_path: str):
        """ Load the model from a file.
        @param model_path (str): path to model
        """
        params = torch.load(model_path, map_location=lambda storage, loc: storage,weights_only=False)
        args = params['args']
        if 'bidirectional' not in args:
            print("WARNING: 'bidirectional' key not found in saved args. Forcing True for decoding.", file=sys.stderr)
            args['bidirectional'] = True
        model = NMT(vocab=params['vocab'], **args)
        model.load_state_dict(params['state_dict'])

        return model

    def save(self, path: str):
        """ Save the odel to a file.
        @param path (str): path to the model
        """
        print('save model parameters to [%s]' % path, file=sys.stderr)

        params = {
            'args': dict(embed_size=self.model_embeddings.embed_size, hidden_size=self.hidden_size, dropout_rate=self.dropout_rate, rnn_layer=self.rnn_layer, num_layers=self.num_layers, activation=self.activation),
            'vocab': self.vocab,
            'state_dict': self.state_dict()
        }

        torch.save(params, path)

In [17]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import sys
from collections import namedtuple
from typing import Dict, List, Set, Tuple, Union

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


# 1. 定义一个 *新的* Hypothesis，它包含 attention_weights 字段
Hypothesis = namedtuple('Hypothesis', ['value', 'score', 'attention_weights'])


class VisualizableNMT(NMT):
    """
    这是一个 NMT 的子类，专门用于推断和可视化。
    它重写了 step 和 beam_search 方法来捕捉注意力权重。
    你原来的 nmt_model.py 文件保持不变。
    """

    # 2. 重写 'step' 方法
    # (这是从 nmt_model.py 完整复制过来的，只修改了最后一行)
    def step(self, Ybar_t: torch.Tensor,
            dec_state: Tuple[torch.Tensor, torch.Tensor],
            enc_hiddens: torch.Tensor,
            enc_hiddens_proj: torch.Tensor,
            enc_masks: torch.Tensor) -> Tuple[Tuple, torch.Tensor, torch.Tensor]:
        """ Compute one forward step of the LSTM decoder, including the attention computation.
        """
        combined_output = None
        dec_state = self.decoder(Ybar_t, dec_state)
        dec_hidden, dec_cell = dec_state
        e_t = torch.squeeze(torch.bmm(enc_hiddens_proj, torch.unsqueeze(dec_hidden, dim=2)), dim=2)

        # Set e_t to -inf where enc_masks has 1 to ignore <pad> tokens
        if enc_masks is not None:
            e_t.data.masked_fill_(enc_masks.bool(), -float('inf'))

        alpha_t = F.softmax(e_t, dim=1) # (b, src_len)
        a_t = torch.squeeze(torch.bmm(torch.unsqueeze(alpha_t, 1), enc_hiddens), dim=1) # (b, 2h)
        U_t = torch.cat((dec_hidden, a_t), dim=1)
        V_t = self.combined_output_projection(U_t)
        O_t = self.dropout(self.activation(V_t))

        combined_output = O_t

        # 【修改点】返回 alpha_t (注意力权重) 而不是 e_t
        return dec_state, combined_output, alpha_t

    # 3. 重写 'beam_search' 方法
    # (这是我们之前讨论过的修改版，用于追踪注意力历史)
    def beam_search(self, src_sent: List[str], beam_size: int=5, max_decoding_time_step: int=70) -> List[Hypothesis]:
        """ Given a single source sentence, perform beam search, yielding translations in the target language.
        @returns hypotheses (List[Hypothesis]): a list of hypothesis, each hypothesis has three fields:
                value: List[str]: the decoded target sentence
                score: float: the log-likelihood of the target sentence
                attention_weights: torch.Tensor: a tensor of shape (tgt_len, src_len)
        """
        src_sents_var = self.vocab.src.to_input_tensor([src_sent], self.device)

        src_encodings, dec_init_vec = self.encode(src_sents_var, [len(src_sent)])
        src_encodings_att_linear = self.att_projection(src_encodings)

        h_tm1 = dec_init_vec
        att_tm1 = torch.zeros(1, self.hidden_size, device=self.device)

        eos_id = self.vocab.tgt['</s>']

        hypotheses = [['<s>']]
        hyp_scores = torch.zeros(len(hypotheses), dtype=torch.float, device=self.device)

        # 新增：用于追踪每个假设的注意力历史
        hyp_att_histories = [[]]

        completed_hypotheses = []

        t = 0
        while len(completed_hypotheses) < beam_size and t < max_decoding_time_step:
            t += 1
            hyp_num = len(hypotheses)

            exp_src_encodings = src_encodings.expand(hyp_num,
                                                     src_encodings.size(1),
                                                     src_encodings.size(2))

            exp_src_encodings_att_linear = src_encodings_att_linear.expand(hyp_num,
                                                                           src_encodings_att_linear.size(1),
                                                                           src_encodings_att_linear.size(2))

            y_tm1 = torch.tensor([self.vocab.tgt[hyp[-1]] for hyp in hypotheses], dtype=torch.long, device=self.device)
            y_t_embed = self.model_embeddings.target(y_tm1)

            x = torch.cat([y_t_embed, att_tm1], dim=-1)

            # 修改：接收 alpha_t
            (h_t, cell_t), att_t, alpha_t  = self.step(x, h_tm1,
                                                      exp_src_encodings, exp_src_encodings_att_linear, enc_masks=None)

            log_p_t = F.log_softmax(self.target_vocab_projection(att_t), dim=-1)

            live_hyp_num = beam_size - len(completed_hypotheses)
            contiuating_hyp_scores = (hyp_scores.unsqueeze(1).expand_as(log_p_t) + log_p_t).view(-1)
            top_cand_hyp_scores, top_cand_hyp_pos = torch.topk(contiuating_hyp_scores, k=live_hyp_num)

            prev_hyp_ids = top_cand_hyp_pos // len(self.vocab.tgt)
            hyp_word_ids = top_cand_hyp_pos % len(self.vocab.tgt)

            new_hypotheses = []
            live_hyp_ids = []
            new_hyp_scores = []
            new_hyp_att_histories = [] # 新增

            for prev_hyp_id, hyp_word_id, cand_new_hyp_score in zip(prev_hyp_ids, hyp_word_ids, top_cand_hyp_scores):
                prev_hyp_id = prev_hyp_id.item()
                hyp_word_id = hyp_word_id.item()
                cand_new_hyp_score = cand_new_hyp_score.item()

                hyp_word = self.vocab.tgt.id2word[hyp_word_id]
                new_hyp_sent = hypotheses[prev_hyp_id] + [hyp_word]

                # 新增：记录注意力历史
                prev_att_history = hyp_att_histories[prev_hyp_id]
                current_att_vector = alpha_t[prev_hyp_id]
                new_att_history = prev_att_history + [current_att_vector]

                if hyp_word == '</s>':
                    # 修改：使用我们新的 Hypothesis tuple
                    completed_hypotheses.append(Hypothesis(value=new_hyp_sent[1:-1],
                                                           score=cand_new_hyp_score,
                                                           attention_weights=torch.stack(new_att_history)))
                else:
                    new_hypotheses.append(new_hyp_sent)
                    live_hyp_ids.append(prev_hyp_id)
                    new_hyp_scores.append(cand_new_hyp_score)
                    new_hyp_att_histories.append(new_att_history) # 新增

            if len(completed_hypotheses) == beam_size:
                break

            live_hyp_ids = torch.tensor(live_hyp_ids, dtype=torch.long, device=self.device)
            h_tm1 = (h_t[live_hyp_ids], cell_t[live_hyp_ids])
            att_tm1 = att_t[live_hyp_ids]

            hypotheses = new_hypotheses
            hyp_scores = torch.tensor(new_hyp_scores, dtype=torch.float, device=self.device)
            # 累积所有decoder输出对所有encoder的注意力
            hyp_att_histories = new_hyp_att_histories # 新增

        if len(completed_hypotheses) == 0:
            # 修改：使用我们新的 Hypothesis tuple
            completed_hypotheses.append(Hypothesis(value=hypotheses[0][1:],
                                                   score=hyp_scores[0].item(),
                                                   attention_weights=torch.stack(hyp_att_histories[0])))

        completed_hypotheses.sort(key=lambda hyp: hyp.score, reverse=True)

        return completed_hypotheses

    # 4. 重写 'load' 静态方法
    @staticmethod
    def load(model_path: str):
        """
        加载模型文件。
        此方法被重写，以确保加载参数时创建的是 VisualizableNMT 实例。
        """
        params = torch.load(model_path, map_location=lambda storage, loc: storage, weights_only=False)
        args = params['args']
        if 'bidirectional' not in args:
            print("WARNING: 'bidirectional' key not found in saved args. Forcing True for decoding.", file=sys.stderr)
            args['bidirectional'] = True

        # 【修改点】创建 *当前类* (VisualizableNMT) 的实例，而不是 NMT
        model = VisualizableNMT(vocab=params['vocab'], **args)
        model.load_state_dict(params['state_dict'])

        return model

In [18]:

import math

import numpy as np
import nltk
import sentencepiece as spm
import jieba

nltk.download('punkt')


def pad_sents(sents, pad_token):
    """ Pad list of sentences according to the longest sentence in the batch.
        The paddings should be at the end of each sentence.
    @param sents (list[list[str]]): list of sentences, where each sentence
                                    is represented as a list of words
    @param pad_token (str): padding token
    @returns sents_padded (list[list[str]]): list of sentences where sentences shorter
        than the max length sentence are padded out with the pad_token, such that
        each sentences in the batch now has equal length.
    """
    sents_padded = []

    ### YOUR CODE HERE (~6 Lines)
    max_len = max([len(each) for each in sents])
    for sent in sents:
        sent += [pad_token] * (max_len - len(sent))
        sents_padded.append(sent)

    ### END YOUR CODE

    return sents_padded

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Usage:
    vocab.py --train-src=<file> --train-tgt=<file> [options] VOCAB_FILE

Options:
    -h --help                  Show this screen.
    --train-src=<file>         File of training source sentences
    --train-tgt=<file>         File of training target sentences
    --size=<int>               vocab size [default: 50000]
    --freq-cutoff=<int>        frequency cutoff [default: 2]
"""

from collections import Counter
from itertools import chain
import json
import torch
from typing import List
import sentencepiece as spm


class VocabEntry(object):
    """ Vocabulary Entry, i.e. structure containing either
    src or tgt language terms.
    """
    def __init__(self, word2id=None):
        """ Init VocabEntry Instance.
        @param word2id (dict): dictionary mapping words 2 indices
        """
        if word2id:
            self.word2id = word2id
        else:
            self.word2id = dict()
            self.word2id['<pad>'] = 0   # Pad Token
            self.word2id['<s>'] = 1     # Start Token
            self.word2id['</s>'] = 2    # End Token
            self.word2id['<unk>'] = 3   # Unknown Token
        self.unk_id = self.word2id['<unk>']
        self.id2word = {v: k for k, v in self.word2id.items()}

    def __getitem__(self, word):
        """ Retrieve word's index. Return the index for the unk
        token if the word is out of vocabulary.
        @param word (str): word to look up.
        @returns index (int): index of word
        """
        return self.word2id.get(word, self.unk_id)

    def __contains__(self, word):
        """ Check if word is captured by VocabEntry.
        @param word (str): word to look up
        @returns contains (bool): whether word is contained
        """
        return word in self.word2id

    def __len__(self):
        """ Compute number of words in VocabEntry.
        @returns len (int): number of words in VocabEntry
        """
        return len(self.word2id)

    def __repr__(self):
        """ Representation of VocabEntry to be used
        when printing the object.
        """
        return 'Vocabulary[size=%d]' % len(self)

    def id2word(self, wid):
        """ Return mapping of index to word.
        @param wid (int): word index
        @returns word (str): word corresponding to index
        """
        return self.id2word[wid]

    def add(self, word):
        """ Add word to VocabEntry, if it is previously unseen.
        @param word (str): word to add to VocabEntry
        @return index (int): index that the word has been assigned
        """
        if word not in self:
            wid = self.word2id[word] = len(self)
            self.id2word[wid] = word
            return wid
        else:
            return self[word]

    def words2indices(self, sents):
        """ Convert list of words or list of sentences of words
        into list or list of list of indices.
        @param sents (list[str] or list[list[str]]): sentence(s) in words
        @return word_ids (list[int] or list[list[int]]): sentence(s) in indices
        """
        if type(sents[0]) == list:
            return [[self[w] for w in s] for s in sents]
        else:
            return [self[w] for w in sents]

    def indices2words(self, word_ids):
        """ Convert list of indices into words.
        @param word_ids (list[int]): list of word ids
        @return sents (list[str]): list of words
        """
        return [self.id2word[w_id] for w_id in word_ids]

    def to_input_tensor(self, sents: List[List[str]], device: torch.device) -> torch.Tensor:
        """ Convert list of sentences (words) into tensor with necessary padding for
        shorter sentences.

        @param sents (List[List[str]]): list of sentences (words)
        @param device: device on which to load the tesnor, i.e. CPU or GPU

        @returns sents_var: tensor of (max_sentence_length, batch_size)
        """
        word_ids = self.words2indices(sents)
        sents_t = pad_sents(word_ids, self['<pad>'])
        sents_var = torch.tensor(sents_t, dtype=torch.long, device=device)
        return torch.t(sents_var)

    @staticmethod
    def from_corpus(corpus, size, freq_cutoff=2):
        """ Given a corpus construct a Vocab Entry.
        @param corpus (list[str]): corpus of text produced by read_corpus function
        @param size (int): # of words in vocabulary
        @param freq_cutoff (int): if word occurs n < freq_cutoff times, drop the word
        @returns vocab_entry (VocabEntry): VocabEntry instance produced from provided corpus
        """
        vocab_entry = VocabEntry()
        word_freq = Counter(chain(*corpus))
        valid_words = [w for w, v in word_freq.items() if v >= freq_cutoff]
        print('number of word types: {}, number of word types w/ frequency >= {}: {}'
              .format(len(word_freq), freq_cutoff, len(valid_words)))
        top_k_words = sorted(valid_words, key=lambda w: word_freq[w], reverse=True)[:size]
        for word in top_k_words:
            vocab_entry.add(word)
        return vocab_entry

    @staticmethod
    def from_subword_list(subword_list):
        vocab_entry = VocabEntry()
        for subword in subword_list:
            vocab_entry.add(subword)
        return vocab_entry


class Vocab(object):
    """ Vocab encapsulating src and target langauges.
    """
    def __init__(self, src_vocab: VocabEntry, tgt_vocab: VocabEntry):
        """ Init Vocab.
        @param src_vocab (VocabEntry): VocabEntry for source language
        @param tgt_vocab (VocabEntry): VocabEntry for target language
        """
        self.src = src_vocab
        self.tgt = tgt_vocab

    @staticmethod
    def build(src_sents, tgt_sents) -> 'Vocab':
        """ Build Vocabulary.
        @param src_sents (list[str]): Source subwords provided by SentencePiece
        @param tgt_sents (list[str]): Target subwords provided by SentencePiece
        """

        print('initialize source vocabulary ..')
        src = VocabEntry.from_subword_list(src_sents)

        print('initialize target vocabulary ..')
        tgt = VocabEntry.from_subword_list(tgt_sents)

        return Vocab(src, tgt)

    def save(self, file_path):
        """ Save Vocab to file as JSON dump.
        @param file_path (str): file path to vocab file
        """
        with open(file_path, 'w') as f:
            json.dump(dict(src_word2id=self.src.word2id, tgt_word2id=self.tgt.word2id), f, indent=2)

    @staticmethod
    def load(file_path):
        """ Load vocabulary from JSON dump.
        @param file_path (str): file path to vocab file
        @returns Vocab object loaded from JSON dump
        """
        entry = json.load(open(file_path, 'r'))
        src_word2id = entry['src_word2id']
        tgt_word2id = entry['tgt_word2id']
        return Vocab(VocabEntry(src_word2id), VocabEntry(tgt_word2id))

    def __repr__(self):
        """ Representation of Vocab to be used
        when printing the object.
        """
        return 'Vocab(source %d words, target %d words)' % (len(self.src), len(self.tgt))


def get_vocab_list(file_path, source, vocab_size):
    """ Use SentencePiece to tokenize and acquire list of unique subwords.
    @param file_path (str): file path to corpus
    @param source (str): tgt or src
    @param vocab_size: desired vocabulary size
    """
    spm.SentencePieceTrainer.train(input=file_path, model_prefix=source, vocab_size=vocab_size)     # train the spm model
    sp = spm.SentencePieceProcessor()                                                               # create an instance; this saves .model and .vocab files
    sp.load('{}.model'.format(source))                                                              # loads tgt.model or src.model
    sp_list = [sp.id_to_piece(piece_id) for piece_id in range(sp.get_piece_size())]                 # this is the list of subwords
    return sp_list

In [20]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
使用说明:
    visualize.py <MODEL_PATH> <VOCAB_FILE> <SENTENCE> [options]
... (省略)
"""

import torch
# 【修改点 1】: 从 visualize_model 导入新类
# from visualize_model import VisualizableNMT, Hypothesis
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import sys
from typing import List

# --- 新增导入 ---
import altair as alt
import pandas as pd
# --- 结束 ---


# --- 中文分词函数 (保持不变) ---
def tokenize_zh(sentence: str) -> List[str]:
    """
    简单的中文分词器（按字分割）。
    !! 如果你使用了BPE或SentencePiece，请务必用你的分词器替换此函数 !!
    """
    print(f"提示：正在使用简单的“按字分割”对源句进行分词...")
    return list(sentence)
# ---------------


def visualize_attention(src_sent_str: str,
                        model: VisualizableNMT,
                        vocab: Vocab,
                        beam_size: int,
                        max_decoding_time_step: int,
                        device: torch.device):
    """
    对单个句子进行推断并使用 Altair 可视化注意力。
    """

    # --- 步骤 1-4：模型推断 (保持不变) ---

    # 1. 预处理源句
    src_tokens = tokenize_zh(src_sent_str)
    src_sent_with_tokens = ['<s>'] + src_tokens + ['</s>']
    print(f"源句 (Tokens): {src_sent_with_tokens}")

    # 2. 设置为评估模式
    model.eval()

    # 3. 运行推断
    with torch.no_grad():
        hypotheses = model.beam_search(src_sent_with_tokens,
                                     beam_size=beam_size,
                                     max_decoding_time_step=max_decoding_time_step)

    # 4. 获取结果
    if not hypotheses:
        print("错误：未能生成翻译。")
        return

    top_hyp = hypotheses[0]
    tgt_tokens = top_hyp.value
    attention_matrix = top_hyp.attention_weights.cpu().numpy()
    attention_matrix_clipped = attention_matrix[:, 1:-1] # 裁掉 <s> 和 </s>

    if len(tgt_tokens) != attention_matrix_clipped.shape[0]:
        print(f"警告: 目标 token 数量 ({len(tgt_tokens)}) 与 "
              f"注意力矩阵高度 ({attention_matrix_clipped.shape[0]}) 不匹配。正在裁剪...")
        min_len = min(len(tgt_tokens), attention_matrix_clipped.shape[0])
        tgt_tokens = tgt_tokens[:min_len]
        attention_matrix_clipped = attention_matrix_clipped[:min_len, :]

    print(f"翻译 (Tokens): {tgt_tokens}")
    print(f"翻译 (句子): {' '.join(tgt_tokens)}")

    # --- 步骤 5：使用 Altair 进行可视化 (替换 matplotlib) ---

    try:
        # 1. 准备 DataFrame
        # attention_matrix_clipped 的形状是 (tgt_len, src_len)
        tgt_len, src_len = attention_matrix_clipped.shape

        data = []
        for i in range(tgt_len):  # 遍历 Y 轴 (目标句)
            for j in range(src_len):  # 遍历 X 轴 (源句)
                data.append({
                    "target_token": tgt_tokens[i],
                    "source_token": src_tokens[j],
                    "attention": attention_matrix_clipped[i, j],
                    "target_index": i,  # 用于排序
                    "source_index": j,  # 用于排序
                })

        df = pd.DataFrame(data)

        # 2. 创建 Altair 热力图

        # 定义X轴，按源句索引排序
        x_axis = alt.X('source_token',
                       sort=alt.EncodingSortField(field="source_index", op="min", order='ascending'),
                       axis=alt.Axis(title="Source Sentence (源句)"))

        # 定义Y轴，按目标句索引排序
        y_axis = alt.Y('target_token',
                       sort=alt.EncodingSortField(field="target_index", op="min", order='ascending'),
                       axis=alt.Axis(title="Generated Translation (目标句)"))

        # 定义颜色
        color = alt.Color('attention',
                          scale=alt.Scale(range='heatmap'), # 使用 'heatmap' 色彩方案
                          legend=alt.Legend(title="Attention Weight"))

        # 定义工具提示 (鼠标悬停时显示)
        tooltip = ['source_token', 'target_token', 'attention']

        # 组合图表
        chart = alt.Chart(df).mark_rect().encode(
            x=x_axis,
            y=y_axis,
            color=color,
            tooltip=tooltip
        ).properties(
            title='Attention Heatmap (zh -> en)'
        ).interactive() # 添加交互性 (缩放和平移)

        # 3. 保存图表
        output_filename = "attention_visualization.html"
        chart.save(output_filename)

        print(f"\n成功！注意力可视化图像已保存到 {output_filename}")
        print(f"请在你的浏览器中打开 {output_filename} 查看交互式图表。")

    except Exception as e:
        print(f"\n使用 Altair 绘图时出错: {e}")
        print("请确保你已安装 'altair' 和 'pandas' (`pip install altair pandas`)")


In [21]:

def read_sent_zh(sent, source):
    """ Read a Chinese sentence, seperate the words using jieba, and generate tokens
    @param sent (str): path to file containing corpus
    @param source (str): "tgt" or "src" for selecting sp model
    """
    sp = spm.SentencePieceProcessor()
    sp.load('{}.model'.format(source))

    sent = " ".join(jieba.cut(sent, HMM=True))
    subword_tokens = sp.encode_as_pieces(sent)

    return subword_tokens


In [22]:
# --- 确保在文件顶部导入了这些 ---
import torch
import sys
from typing import Dict, List
import os
import sacrebleu
# ---

def decode_single_sentence(model_path: str, vocab_path: str, sentence: str, use_cuda: bool=True,reference_sentence: str = None,  beam_size_val: int = 10, max_steps: int = 70):
    """
    加载模型并对单个句子执行解码（推断）。

    此版本 *复用* 了项目中的 'read_sent_zh' 函数 (来自 utils.py)
    来确保与训练时 *完全一致* 的分词。

    @param model_path (str): 模型文件路径 (e.g., "model.bin")
    @param vocab_path (str): 词汇表文件路径 (e.g., "vocab.json")
    @param sentence (str): 要翻译的中文句子 (e.g., "你好")
    @param use_cuda (bool): 是否使用 GPU
    @param beam_size_val (int): 集束搜索大小
    @param max_steps (int): 最大解码步数
    @returns str: 翻译后的句子
    """

    # --- 1. 加载模型 (来自原版 decode) ---
    print(f"load model from {model_path}", file=sys.stderr)
    model = VisualizableNMT.load(model_path)

    # --- 2. 加载词汇表 (处理单句时必须) ---
    print(f"load vocab from {vocab_path}", file=sys.stderr)
    vocab = Vocab.load(vocab_path)
    model.vocab = vocab  # 确保模型实例有词汇表

    # --- 3. 设置设备 (来自原版 decode) ---
    if use_cuda:
        model = model.to(torch.device("cuda:0"))
    model.eval() # 设为评估模式

    # --- 4. 处理单句 (使用 read_sent_zh) ---
    #
    # 这是关键：我们调用 'read_sent_zh' 来获取
    # 和 'read_corpus' 完全一致的分词。
    #
    # 我们从原版 decode 函数中看到 read_corpus
    # 被调用时使用了 source='src'。
    # 我们假设 read_sent_zh 也需要这个 'source' 参数。
    #
    print("Tokenizing sentence using 'read_sent_zh'...", file=sys.stderr)
    src_tokens = read_sent_zh(sentence, source='src')

    print(f"Source (Tokens): {src_tokens}", file=sys.stderr)

    # (注意：我们 *不* 在这里手动添加 '<s>' 或 '</s>')
    # (nmt_model.py 中的 model.beam_search 会通过
    #  self.vocab.src.to_input_tensor 来处理它们)

    # --- 5. 执行推断 (来自原版 decode) ---
    # 我们直接调用 *model* 上的 beam_search 方法 (来自 nmt_model.py)
    with torch.no_grad():
        # model.beam_search 接收一个 List[str]
        # (不带 <s>, </s> 的 token 列表)
        hypotheses = model.beam_search(src_tokens,
                                     beam_size=beam_size_val,
                                     max_decoding_time_step=max_steps)

    # --- 6. 格式化输出 (来自原版 decode 的 with open 循环) ---
    if not hypotheses:
        print("Error: No hypotheses generated.", file=sys.stderr)
        return None

    top_hyp = hypotheses[0]

    # 从 (value, score, ...) 中提取 'value'
    hyp_sent = ""
    try:
        # 兼容 (value, score, attention_weights)
        raw_hyp_sent = ''.join(top_hyp.value)
        hyp_sent = raw_hyp_sent.replace(' ', ' ').replace('▁', ' ')
        hyp_sent = hyp_sent.strip()
    except AttributeError:
        # 兼容 (value, score)
        raw_hyp_sent = ''.join(top_hyp[0])
        hyp_sent = raw_hyp_sent.replace(' ', ' ')
        hyp_sent = hyp_sent.strip()

    if reference_sentence:
        # sacrebleu.sentence_bleu 需要一个 str (模型翻译)
        # 和一个 str 列表 (标准答案可以有多个)
        bleu = sacrebleu.sentence_bleu(hyp_sent, [reference_sentence])

        # .score 给出的是 0-100 的分数
        print(f"BLEU 分数 (对比参考): {bleu.score:.2f}")
    print("\n--- 翻译结果 ---")
    print(f"源句: {sentence}")
    print(f"译句: {hyp_sent}")
    print(f"得分 (LogProb): {top_hyp.score:.4f}")

    return hyp_sent

In [23]:
!pip install sacrebleu

In [24]:
sentence="尽管 信任度 差距 在 近几年 中 有所 缩小 ， 但 即使 最近 的 衰退   —   —   这 通常 被 归咎于 欧盟 所 实施 的 紧缩 和 欧元区 危机   —   —   也 只是 略微 降低 了 欧洲议会 对 国家 议会 的 优势 。"
reference_text = "Although the trust gap has narrowed in recent years, even the recent recession — which is often blamed on EU-imposed austerity and the eurozone crisis — has only slightly reduced the European Parliament's advantage over national parliaments."
decode_single_sentence('model.bin','vocab_zh_en.json',sentence=sentence,reference_sentence=reference_text)

load model from model.bin
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
load vocab from vocab_zh_en.json
Tokenizing sentence using 'read_sent_zh'...
Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.772 seconds.
DEBUG:jieba:Loading model cost 0.772 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.
Source (Tokens): ['▁尽管', '▁信任', '度', '▁差距', '▁在', '▁近几年', '▁中', '▁有所', '▁缩小', '▁,', '▁但', '▁即使', '▁最', '近', '▁的', '▁衰退', '▁—', '▁—', '▁这', '▁通', '常', '▁被', '▁归咎', '于', '▁欧盟', '▁所', '▁实施', '▁的', '▁紧缩', '▁和', '▁欧元区', '▁危机', '▁—', '▁—', '▁也', '▁只', '是', '▁略', '微', '▁降低', '▁了', '▁欧洲', '议会', '▁对', '▁国家', '▁议会', '▁的', '▁优势', '▁。']


torch.Size([1, 49, 1024])
****************************************
BLEU 分数 (对比参考): 42.68

--- 翻译结果 ---
源句: 尽管 信任度 差距 在 近几年 中 有所 缩小 ， 但 即使 最近 的 衰退   —   —   这 通常 被 归咎于 欧盟 所 实施 的 紧缩 和 欧元区 危机   —   —   也 只是 略微 降低 了 欧洲议会 对 国家 议会 的 优势 。
译句: While the trust gap has narrowed in recent years, even the recent recession – often blamed on the austerity and eurozone crisis pursued by the European Union – only slightly lowered the European Parliament’s strength to national parliaments.
得分 (LogProb): -21.2387


'While the trust gap has narrowed in recent years, even the recent recession – often blamed on the austerity and eurozone crisis pursued by the European Union – only slightly lowered the European Parliament’s strength to national parliaments.'

In [25]:
# --- 导入可视化库 ---
import altair as alt
import pandas as pd
import sacrebleu

# 启用 Colab 的 Altair 渲染器
alt.renderers.enable('colab')

def visualize_attention(
        model_path: str,
        vocab_path: str,
        sentence: str,
        use_cuda: bool,
        reference_sentence: str = None,
        beam_size: int = 10,
        max_steps: int = 70
    ):
    """
    加载模型，对单个句子进行推断，并使用 Altair 可视化注意力。
    在 Colab 中，此函数将返回一个可显示的 chart 对象。
    """

    # --- 1. 加载模型 ---
    print(f"load model from {model_path}", file=sys.stderr)
    model = VisualizableNMT.load(model_path)

    # --- 2. 加载词汇表 ---
    print(f"load vocab from {vocab_path}", file=sys.stderr)
    vocab = Vocab.load(vocab_path)
    model.vocab = vocab

    # --- 3. 设置设备 ---
    if use_cuda:
        model = model.to(torch.device("cuda:0"))
    model.eval()

    # --- 4. 处理单句 (使用 read_sent_zh) ---
    print("Tokenizing sentence using 'read_sent_zh'...", file=sys.stderr)
    src_tokens = read_sent_zh(sentence, source='src')
    print(f"Source (Tokens): {src_tokens}", file=sys.stderr)

    # --- 5. 执行推断 ---
    with torch.no_grad():
        hypotheses = model.beam_search(src_tokens,
                                     beam_size=beam_size,
                                     max_decoding_time_step=max_steps)

    # --- 6. 提取数据 ---
    if not hypotheses:
        print("Error: No hypotheses generated.", file=sys.stderr)
        return None

    top_hyp = hypotheses[0]
    tgt_tokens = top_hyp.value
    # 得到全局注意力值
    attention_matrix = top_hyp.attention_weights.cpu().numpy()
    attention_matrix_clipped = attention_matrix[:, 1:-1] # 裁掉 <s> 和 </s>

    # --- 7. 打印翻译结果 (使用清理逻辑) ---
    raw_hyp_sent = ''.join(top_hyp.value)
    hyp_sent = raw_hyp_sent.replace(' ', ' ').replace('▁', ' ').strip()

    print("\n--- 翻译结果 ---")
    print(f"源句: {sentence}")
    print(f"译句: {hyp_sent}")
    print(f"得分 (LogProb): {top_hyp.score:.4f}")

    if reference_sentence:
        bleu = sacrebleu.sentence_bleu(hyp_sent, [reference_sentence])
        print(f"BLEU 分数 (对比参考): {bleu.score:.2f}")

    # --- 8. 准备 DataFrame 并创建 Altair 图表 ---
    try:
        tgt_len, src_len = attention_matrix_clipped.shape

        # 维度检查
        if len(tgt_tokens) != tgt_len:
            min_len = min(len(tgt_tokens), tgt_len)
            tgt_tokens = tgt_tokens[:min_len]
            attention_matrix_clipped = attention_matrix_clipped[:min_len, :]
            tgt_len = min_len

        if len(src_tokens) != src_len:
             min_len_src = min(len(src_tokens), src_len)
             src_tokens = src_tokens[:min_len_src]
             attention_matrix_clipped = attention_matrix_clipped[:, :min_len_src]
             src_len = min_len_src

        data = []
        for i in range(tgt_len):  # Y 轴 (目标句)
            for j in range(src_len):  # X 轴 (源句)
                data.append({
                    "target_token": tgt_tokens[i].replace('▁', ' '),
                    "source_token": src_tokens[j].replace('▁', ' '),
                    "attention": attention_matrix_clipped[i, j],
                    "target_index": i,  # 用于排序
                    "source_index": j,  # 用于排序
                })

        df = pd.DataFrame(data)

        x_axis = alt.X('source_token',
                       sort=alt.EncodingSortField(field="source_index", op="min", order='ascending'),
                       axis=alt.Axis(title="Source Sentence (源句)"))

        y_axis = alt.Y('target_token',
                       sort=alt.EncodingSortField(field="target_index", op="min", order='ascending'),
                       axis=alt.Axis(title="Generated Translation (目标句)"))

        color = alt.Color('attention',
                          scale=alt.Scale(range='heatmap'),
                          legend=alt.Legend(title="Attention Weight"))

        tooltip = [
            alt.Tooltip('source_token', title='Source'),
            alt.Tooltip('target_token', title='Target'),
            alt.Tooltip('attention', title='Weight', format=".4f")
        ]

        chart = alt.Chart(df).mark_rect().encode(
            x=x_axis,
            y=y_axis,
            color=color,
            tooltip=tooltip
        ).properties(
            title='Attention Heatmap (zh -> en)'
        ).interactive()

        print(f"\n✅ 图表生成成功！")

        # 【关键】返回图表对象
        return chart

    except Exception as e:
        print(f"\n使用 Altair 绘图时出错: {e}", file=sys.stderr)
        return None

print("✅ 'visualize_attention' 函数定义成功。")

✅ 'visualize_attention' 函数定义成功。


In [26]:
MODEL_FILE = "model.bin"
VOCAB_FILE = "vocab_zh_en.json" # 或 'vocab.pt'

# 你想要翻译的中文句子
SENTENCE_TO_TRANSLATE = "尽管 信任度 差距 在 近几年 中 有所 缩小 ， 但 即使 最近 的 衰退 — — 这 通常 被 归咎于 欧盟 所 实施 的 紧缩 和 欧元区 危机 — — 也 只是 略微 降低 了 欧洲议会 对 国家 议会 的 优势 。"

# (可选) 提供一个标准答案来计算 BLEU
REFERENCE_TRANSLATION = "Although the trust gap has narrowed in recent years, even the recent recession -- which is often blamed on EU-imposed austerity and the eurozone crisis -- has only slightly reduced the European Parliament's advantage over national parliaments."
# REFERENCE_TRANSLATION = None  # 如果没有标准答案，请设为 None

# 自动检测 CUDA
USE_CUDA = torch.cuda.is_available()

if USE_CUDA:
    print("🚀 正在使用 GPU (CUDA) 运行...")
else:
    print("🐌 正在使用 CPU 运行...")

# --- 调用函数 ---
chart = visualize_attention(
    model_path=MODEL_FILE,
    vocab_path=VOCAB_FILE,
    sentence=SENTENCE_TO_TRANSLATE,
    use_cuda=USE_CUDA,
    reference_sentence=REFERENCE_TRANSLATION,
    beam_size=10,
    max_steps=100  # 增加了最大步数以防句子太长
)

# --- 在 Colab 中显示图表 ---
# (只需将 chart 作为单元格的最后一行)
if chart:
    chart
else:
    print("❌ 未能生成图表。")

🚀 正在使用 GPU (CUDA) 运行...


load model from model.bin


torch.Size([1, 49, 1024])
****************************************

--- 翻译结果 ---
源句: 尽管 信任度 差距 在 近几年 中 有所 缩小 ， 但 即使 最近 的 衰退 — — 这 通常 被 归咎于 欧盟 所 实施 的 紧缩 和 欧元区 危机 — — 也 只是 略微 降低 了 欧洲议会 对 国家 议会 的 优势 。
译句: While the trust gap has narrowed in recent years, even the recent recession – often blamed on the austerity and eurozone crisis pursued by the European Union – only slightly lowered the European Parliament’s strength to national parliaments.
得分 (LogProb): -21.2387
BLEU 分数 (对比参考): 42.68

✅ 图表生成成功！


load vocab from vocab_zh_en.json
Tokenizing sentence using 'read_sent_zh'...
Source (Tokens): ['▁尽管', '▁信任', '度', '▁差距', '▁在', '▁近几年', '▁中', '▁有所', '▁缩小', '▁,', '▁但', '▁即使', '▁最', '近', '▁的', '▁衰退', '▁—', '▁—', '▁这', '▁通', '常', '▁被', '▁归咎', '于', '▁欧盟', '▁所', '▁实施', '▁的', '▁紧缩', '▁和', '▁欧元区', '▁危机', '▁—', '▁—', '▁也', '▁只', '是', '▁略', '微', '▁降低', '▁了', '▁欧洲', '议会', '▁对', '▁国家', '▁议会', '▁的', '▁优势', '▁。']


In [27]:
chart

alt.Chart(...)